# Deep Learning

### DL imports

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

### Misc. Imports

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import json
import os

## Get data from Kaggle

In [ ]:
kaggle_username = input("Insert Kaggle username: ")

In [ ]:
kaggle_key = input("Insert Kaggle key: ")

In [ ]:
os.environ['KAGGLE_USERNAME'] = kaggle_username
os.environ['KAGGLE_KEY'] = kaggle_key

In [ ]:
!kaggle datasets download -d yelp-dataset/yelp-dataset

In [ ]:
!mkdir yelp-data
!mv yelp-dataset.zip yelp-data
!unzip yelp-dataset.zip

## Examine and plot it

## Find correct architecture

## Train

## Evaluate